In [18]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import osmnx as ox
import networkx as nx
from scipy.optimize import linear_sum_assignment
from geopy.distance import geodesic

In [19]:
def get_shortest_path_length(G, start_point, end_point):
    start_node = ox.distance.nearest_nodes(G, start_point[1], start_point[0])
    end_node = ox.distance.nearest_nodes(G, end_point[1], end_point[0])
    length = nx.shortest_path_length(G, start_node, end_node, weight='length')
    return length / 1000  # Возвращаем длину в километрах

In [20]:
def calculate_travel_time(distance, speed=40):
    return distance / speed  # Время в часах

In [21]:
def get_brigade_coords(brigade_name, incident_date):
    if brigade_name == 'D' and incident_date >= datetime(2023, 12, 15):
        return brigades['D_new']
    else:
        return brigades[brigade_name]

In [22]:
def adjust_start_time_for_shift_change(incident_time, brigade_name):
    shift_start_hour = 9 if brigade_name in ['A', 'C'] else 8
    shift_change_time = datetime(incident_time.year, incident_time.month, incident_time.day, shift_start_hour)
    if incident_time < shift_change_time:
        return max(incident_time, shift_change_time + timedelta(minutes=30))
    else:
        return incident_time + timedelta(minutes=30)

In [23]:
brigades = {
    "A": (55.818119, 37.578991),
    "B": (55.753185, 37.410267),
    "C": (55.793629, 37.708969),
    "D": (55.638885, 37.761070),
    "D_new": (55.837784, 37.655743)
}

In [24]:
incidents = pd.read_excel('2023new.xlsx')
incidents['Широта'] = pd.to_numeric(incidents['Широта'], errors='coerce')
incidents['Долгота'] = pd.to_numeric(incidents['Долгота'], errors='coerce')
incidents.dropna(subset=['Широта', 'Долгота'], inplace=True)
incidents['Т события'] = pd.to_datetime(incidents['Т события'])

In [25]:
G = ox.graph_from_place('Moscow, Russia', network_type='drive')
G = ox.project_graph(G)
G = nx.convert_node_labels_to_integers(G)

In [26]:
# Типо матрица затрат
def calculate_cost_matrix(incidents, brigades, G):
    cost_matrix = []
    for _, incident in incidents.iterrows():
        costs = []
        incident_coord = (incident['Широта'], incident['Долгота'])
        for brigade_name in brigades.keys():
            brigade_coord = get_brigade_coords(brigade_name, incident['Т события'])
            path_length = get_shortest_path_length(G, brigade_coord, incident_coord)
            travel_time_hours = calculate_travel_time(path_length)
            costs.append(travel_time_hours)
        cost_matrix.append(costs)
    return np.array(cost_matrix)

In [27]:
def assign_incidents_kuhn_munkres(incidents, brigades, G):
    cost_matrix = calculate_cost_matrix(incidents, brigades, G)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    assignments = [(incidents.iloc[row]['№ МИ'], list(brigades.keys())[col]) for row, col in zip(row_ind, col_ind)]
    return assignments

In [28]:
def distribute_and_save_assignments(incidents, brigades, start_date, end_date, G, filename='assignments_kuhn_munkres.csv'):
    all_assignments = []
    for single_date in pd.date_range(start=start_date, end=end_date):
        daily_incidents = incidents[incidents['Т события'].dt.date == single_date.date()]
        if not daily_incidents.empty:
            daily_assignments = assign_incidents_kuhn_munkres(daily_incidents, brigades, G)
            for incident_id, brigade in daily_assignments:
                all_assignments.append({
                    'Date': single_date.strftime('%Y-%m-%d'),
                    'Incident ID': incident_id,
                    'Assigned Brigade': brigade
                })

    assignments_df = pd.DataFrame(all_assignments)
    assignments_df.to_csv(filename, index=False)
    print(f"Saved assignments to {filename}")

In [29]:
start_date = incidents['Т события'].min()
end_date = incidents['Т события'].max()
distribute_and_save_assignments(incidents, brigades, start_date, end_date, G)

KeyboardInterrupt: 